# Notebook 2: Data Preprocessing
This notebook performs the following tasks:
1. Loads raw stock data from the SQLite database.
2. Cleans and filters the data for inconsistencies and missing values.
3. Engineers new features to enrich the dataset for machine learning.
4. Stores the processed data back into the SQLite database.


## Importing Necessary Libraries
Libraries used:
- `pandas`: For data manipulation and cleaning.
- `numpy`: For numerical operations.
- `sqlite3`: For database interaction.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import sqlite3

## Loading Raw Stock Data
Loads the raw stock data from the SQLite database (`stocks_data.db`) using SQL queries. The data is retrieved from the `stocks` table created in Notebook 1.

In [2]:
# Path to SQLite database
db_path = 'database/stocks_data.db'

# Step 1: Load raw stock data from SQLite
with sqlite3.connect(db_path) as conn:
    query = "SELECT * FROM stocks"
    raw_data = pd.read_sql(query, conn)

print(f"Loaded data from SQLite: {raw_data.shape[0]} rows")


Loaded data from SQLite: 24168 rows


## Cleaning and Validating Data
Steps taken to ensure data quality:
- Converts the `Adj Close` column to numeric format to handle any non-numeric values.
- Drops rows with missing values in the `Adj Close` column.

In [3]:
# Ensure 'Adj Close' is numeric
raw_data['Adj Close'] = pd.to_numeric(raw_data['Adj Close'], errors='coerce')
raw_data = raw_data.dropna(subset=['Adj Close'])

## Feature Engineering
New features are added to improve the dataset for machine learning:
1. **Moving Averages**:
   - `7-day MA`: 7-day rolling average of adjusted close prices.
   - `14-day MA`: 14-day rolling average of adjusted close prices.
2. **Volatility**:
   - Standard deviation of daily percentage changes over a 7-day rolling window.
3. **Lagged Prices**:
   - `Lag_1`: Previous day's adjusted close price.
   - `Lag_2`: Adjusted close price from two days prior.

Missing values resulting from rolling windows and lags are dropped.


In [4]:
# Step 2: Engineer Features
# 2.1 Moving Averages
raw_data['7-day MA'] = raw_data.groupby('Ticker')['Adj Close'].transform(lambda x: x.rolling(window=7).mean())
raw_data['14-day MA'] = raw_data.groupby('Ticker')['Adj Close'].transform(lambda x: x.rolling(window=14).mean())

# 2.2 Volatility (7-day rolling standard deviation of percentage changes)
raw_data['Daily Return'] = raw_data.groupby('Ticker')['Adj Close'].pct_change()  # Daily percentage changes
raw_data['Volatility'] = raw_data.groupby('Ticker')['Daily Return'].transform(lambda x: x.rolling(window=7).std())

# 2.3 Lagged Prices
raw_data['Lag_1'] = raw_data.groupby('Ticker')['Adj Close'].shift(1)
raw_data['Lag_2'] = raw_data.groupby('Ticker')['Adj Close'].shift(2)

# Drop rows with NaN values caused by rolling/lags
processed_data = raw_data.dropna()

## Storing Processed Data
The cleaned and enriched dataset is saved back to the SQLite database in a new table: `processed_stocks`. This allows seamless integration with the next stage (machine learning).


In [5]:
# Step 3: Save processed data back to SQLite
with sqlite3.connect(db_path) as conn:
    processed_data.to_sql('processed_stocks', conn, if_exists='replace', index=False)

print("Processed data saved to SQLite database (table: 'processed_stocks').")

Processed data saved to SQLite database (table: 'processed_stocks').


## Verifying Processed Data
Displays a preview of the processed dataset to confirm:
- The presence of newly engineered features.
- The absence of missing or invalid data.


In [6]:
# Check the processed data
print(processed_data.head())

                   Date  Adj Close      Close       High        Low  \
13  2015-01-22 00:00:00  60.068405  92.870003  92.970001  91.320000   
14  2015-01-23 00:00:00  58.787758  90.889999  92.480003  90.790001   
15  2015-01-26 00:00:00  59.350445  91.760002  91.940002  90.470001   
16  2015-01-27 00:00:00  58.826534  90.949997  91.599998  90.599998   
17  2015-01-28 00:00:00  56.886143  87.949997  90.930000  87.820000   

         Open    Volume      Company Ticker   7-day MA  14-day MA  \
13  92.309998  13559000  Exxon Mobil    XOM  58.734148  58.857040   
14  92.279999  14706300  Exxon Mobil    XOM  58.816386  58.767414   
15  90.610001  10672500  Exxon Mobil    XOM  59.003032  58.835326   
16  91.239998  12301500  Exxon Mobil    XOM  59.186907  58.887994   
17  90.879997  17829700  Exxon Mobil    XOM  58.893997  58.760021   

    Daily Return  Volatility      Lag_1      Lag_2  
13      0.010885    0.011294  59.421597  58.917099  
14     -0.021320    0.014743  60.068405  59.421597  